# Notebook with random quick tests.

RS20250619

## Tool calling with Mistral and Ollama

In [ ]:
import requests
import json

# Define your custom tools
def get_weather(location):
    """Get weather information for a location (mock implementation)"""
    # This is a mock function - in reality you'd call a weather API
    weather_data = {
        "New York": {"temperature": "22°C", "condition": "Sunny"},
        "London": {"temperature": "15°C", "condition": "Cloudy"},
        "Tokyo": {"temperature": "28°C", "condition": "Rainy"}
    }
    return weather_data.get(location, {"temperature": "Unknown", "condition": "Unknown"})

def calculate(expression):
    """Safely calculate a mathematical expression"""
    try:
        # Only allow safe mathematical operations
        allowed_chars = set('0123456789+-*/.() ')
        if all(c in allowed_chars for c in expression):
            result = eval(expression)
            return {"result": result}
        else:
            return {"error": "Invalid characters in expression"}
    except Exception as e:
        return {"error": str(e)}

# Tool definitions for the model
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_weather",
            "description": "Get current weather information for a specific location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city or location to get weather for"
                    }
                },
                "required": ["location"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Calculate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {
                        "type": "string",
                        "description": "Mathematical expression to calculate (e.g., '2 + 2', '10 * 5')"
                    }
                },
                "required": ["expression"]
            }
        }
    }
]

def call_ollama_with_tools(prompt, tools, model_name="mistral"):
    """Call Ollama with tool calling capability"""
    
    url = "http://localhost:11434/api/chat"
    
    payload = {
        "model": model_name,
        "messages": [
            {
                "role": "user",
                "content": prompt
            }
        ],
        "tools": tools,
        "stream": False
    }
    
    try:
        response = requests.post(url, json=payload)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error calling Ollama: {e}")
        return None

def execute_tool_call(tool_call):
    """Execute a tool call and return the result"""
    function_name = tool_call["function"]["name"]
    arguments = tool_call["function"]["arguments"]
    
    if function_name == "get_weather":
        return get_weather(arguments["location"])
    elif function_name == "calculate":
        return calculate(arguments["expression"])
    else:
        return {"error": f"Unknown function: {function_name}"}

def chat_with_tools(prompt, model_name="mistral"):
    """Complete chat interaction with tool calling"""
    print(f"User: {prompt}")
    
    # Initial call to the model
    response = call_ollama_with_tools(prompt, tools, model_name)
    
    if not response:
        return
    
    message = response["message"]
    
    # Check if the model wants to call any tools
    if "tool_calls" in message and message["tool_calls"]:
        print(f"Assistant: I need to use some tools to help you.")
        
        # Execute each tool call
        tool_results = []
        
        for tool_call in message["tool_calls"]:
            print(f"Calling tool: {tool_call['function']['name']}")
            result = execute_tool_call(tool_call)
            id += 1
            tool_results.append({
                "call": tool_call,
                "result": result,
                "id": id
            })
        
        # Make a follow-up call with tool results
        messages = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": message.get("content", ""), "tool_calls": message["tool_calls"]},
        ]
        
        # Add tool results
        for tool_result in tool_results:
            messages.append({
                "role": "tool",
                "content": json.dumps(tool_result["result"]),
                "tool_call_id": tool_result["call"]["id"]
            })
        
        # Call the model again with tool results
        final_payload = {
            "model": model_name,
            "messages": messages,
            "tools": tools,
            "stream": False
        }
        
        final_response = requests.post("http://localhost:11434/api/chat", json=final_payload)
        if final_response.status_code == 200:
            final_message = final_response.json()["message"]
            print(f"Assistant: {final_message['content']}")
        else:
            print("Error in follow-up call")
    else:
        # No tools needed, just return the response
        print(f"Assistant: {message['content']}")

# Test the tool calling functionality
if __name__ == "__main__":
    print("Testing Ollama Tool Calling")
    print("=" * 40)
    
    # Test weather tool
    chat_with_tools("What's the weather like in New York?")
    print()
    
    # Test calculator tool
    chat_with_tools("What is 15 * 7 + 23?")
    print()
    
    # Test multiple tools in one query
    chat_with_tools("What's the weather in London and what is 100 divided by 4?")
    print()
    
    # Test without tools
    chat_with_tools("Tell me a joke about programming")

Testing Ollama Tool Calling
User: What's the weather like in New York?
Assistant: I need to use some tools to help you.
Calling tool: get_weather


KeyError: 'id'